# Introducción

Desde su lanzamiento en Enero de 2008 [pandas](https://pandas.pydata.org/) ha sido la librería de análisis tabular más utilizada dentro del ecosistema Python.

Pandas es una librería muy versátil y con mucho soporte por parte de la comunidad de Data Science.

[scikit-learn](https://scikit-learn.org/stable/), [xgboost](https://xgboost.readthedocs.io/en/stable/) y otras muchas herramientas son compatibles con pandas lo que aumenta su popularidad [(~47k estrellas en GitHub)](https://github.com/pandas-dev/pandas) y también extiende su funcionalidad. **No obstante pandas nació con algunas limitaciones importantes que complican su uso para hacer análisis de datasets masivos (aquellos que no caben en la RAM de tu ordenador).**

En los últimos años la librería [polars](https://pola.rs/) ha ganado mucha notoriedad [(~36k estrellas en GitHub)](https://github.com/pola-rs/polars).

Dentro del workshop de hoy vamos a hacer una **breve introducción a polars y lo vamos a comparar con pandas.**

El dataset utilizado para el análisis es [New Yort Taxi and Limousine Comission Tip Dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Para descargar el dataset, podéis ejecutar el código del script `download_mp.py` y para limpiar los datasets podéis utilizar el notebook [Getting_and_Cleaning_Data.ipynb](./Getting_and_Cleaning_Data.ipynb)

# Los "pecados capitales" de pandas.

El creador de pandas Wes McKinney escribió en detalle sobre algunas de las limitaciones que tiene pandas.

Os recomiendo que lean su blog: [Apache Arrow and the “10 Things I Hate About pandas”](https://wesmckinney.com/blog/apache-arrow-pandas-internals/).

Básicamente los principales problemas de pandas según su creador son:

1. **Internals too far from “the metal”.**<br>
1. No support for memory-mapped datasets.<br>
1. Poor performance in database and file ingest/export.<br>
1. Warty missing data support.<br>
1. **Lack of transparency into memory use, RAM management.**<br>
1. Weak support for categorical data.<br>
1. **Complex groupby operations awkward and slow.**<br>
1. Appending data to a DataFrame tedious and very costly.<br>
1. Limited, non-extensible type metadata.<br>
1. **Eager evaluation model, no query planning.**<br>
1. **“Slow”, limited multicore algorithms for large datasets.**<br>

Dentro de este workshop vamos a hablar un poco más en detalle sobre los 5 puntos marcados en **negrita.**

***Nota: el blog anterior fue escrito antes de la aparición de polars en 2020, pero curiosamente polars resuelve algunas de las problemas que se mencionan en el blog.***

---

El punto número 1, **Internals too far from “the metal** se puede resumir en: pandas está escrito en Python y usa como backend numpy (recientemente pandas también ha añadido la posibilidad de usar Arrow como backend).

Esto hace que cuando tenemos un DataFrame **mixto** (strings, números o hasta objetos python en el DataFrame) las operaciones son mucho más lentas.

`In pandas, an array of strings is an array of PyObject pointers, and the actual string data lives inside PyBytes or PyUnicode structs that live all over the process heap.`

---

Pandas consume mucha memoria interna porque en determinas situaciones debe hacer una copia del DataFrame. Si tenemos muchas columnas esto es algo muy ineficiente.

Según el propio Wes McKinney, `pandas rule of thumb: have 5 to 10 times as much RAM as the size of your dataset`.

### Es decir, si nuestro dataset en memoria ocupa 10GB, es recomendable tener entre 50-100GB de RAM para poder trabajar con pandas sin sustos.

---

**Los puntos 7 y 11** están muy relacionados y tienen que ver con que pandas "carga" todo el dataset en memoria para hacer sus cálculos. Lo anterior junto con el **GIL (Global Interpreter Lock)** de Python hacen muy díficil paralelizar operaciones para ganar velocidad por un lado y procesar datasets que no caben en la RAM por el otro lado.

---

Por último pandas únicamente dispone de **Eager mode**. Esto quiere decir que el código se ejecuta según se encuentra. Al no tener un "motor" de ejecución, no hay espacio para optimizar el código (query planner).

# ¿Que es Polars?

Polars es una librería que nació en 2020 y creada por [Ritchie Vink](https://www.ritchievink.com/blog/2021/02/28/i-wrote-one-of-the-fastest-dataframe-libraries/).

Algunas de las diferencias que tiene respecto a pandas son:

1. Está escrita en **Rust**, un lenguaje de programación muy rápido y uno de los más querido según el [stackoverflow survery de 2023](https://survey.stackoverflow.co/2023/).

2. Polars utiliza desde el inicio **Apache Arrow** como forma de representar los datos. Esto hace que el acceso a los datos sea muy rápido y eficiente.

3. Rust permite escribir **código que se ejecuta en paralelo** de manera muy fácil y polars se beneficia enormemente de ello. La mayoría de las operaciones en polars se ejecutan en diferentes threads y se gana mucha velocidad.

4. Polars es según su creador **"un motor de queries con una interfaz de DataFrames"**. Esto implica que tiene dos modos de ejecutar el código: [Eager y Lazy mode](https://docs.pola.rs/user-guide/concepts/lazy-vs-eager/).

**Eager mode al igual que en pandas: el código se ejecuta según se encuentra. No se realiza ninguna optimización de código.**

**Lazy mode por el otro lado permite al motor de ejecución hacer diferentes optimizaciones sobre el código antes de ejecutar la query y por este motivo puede llegar a ser mucho más rápido y eficiente.**

Resumen de las dos herramientas:

|Feature   |Polars   |Pandas   |
|---|---|---|
|Lenguaje   |Rust   |Python   |
|Backend   |Arrow   |Numpy o PyArrow|
|Velocidad   |Muy rápido gracias a Rust, Multicore Compute y Query Planner   |Más rápido que Python puro   |
|Gestión Memoria|Muy eficiente por Rust y cero copias|Copy on Write|
|Lazy Mode|Soportado|No|
|Multicore|Sí by default|No. Tengo que utilizar otras librerías (p.e: Dask)|
|Datasets Masivos|Sí by default|No. Tengo que utilizar otras librerías (p.e: Dask)|
|Facilidad de uso|Intermedio (aprender nueva síntaxis)|Fácil (ya están familiarizados con la herramienta)|
|Comunidad y soporte|Librería nueva con creciente comunidad|Mucha documentación y ejemplos en internet|
|Ciclos de desarrollo|Rápidos|Largos|

# Imports

In [1]:
# ! pip install hvplot

In [2]:
import warnings

warnings.filterwarnings("ignore")

import os


import pandas as pd
import polars as pl

# para hacer los plots con polars
import hvplot.polars

pd.options.display.float_format = "{:.3f}".format
pl.Config.set_float_precision(3);

In [3]:
print(pl.__version__)

0.20.26


In [4]:
print(hvplot.__version__)

0.10.0


In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

# GLOBAL_VARS

In [19]:
CWD = os.getcwd()
PATH_INPUT_FOLDER = "/Users/nicolaepopescul/code/polanyt/output"

In [20]:
PATH_INPUT_FOLDER

'/Users/nicolaepopescul/code/polanyt/output'

In [21]:
# si estamos en colab
# especificar la ruta donde esta el fichero "yellow_tripdata_2024-01.parquet"
# por ejemplo "/content/drive/MyDrive/POLARS/input"
# PATH_INPUT_FOLDER = ""

In [22]:
TRIP2401 = os.path.join(PATH_INPUT_FOLDER, "yellow_tripdata_2024-01.parquet")

In [23]:
fs = [
    f
    for f in os.listdir(PATH_INPUT_FOLDER)
    if f not in [".ipynb_checkpoints", ".DS_Store"]
]

In [24]:
print(
    f"Tenemos un total de {len(fs)} ficheros dentro de nuestra carpeta\n{PATH_INPUT_FOLDER}"
)

Tenemos un total de 59 ficheros dentro de nuestra carpeta
/Users/nicolaepopescul/code/polanyt/output


# Pandas vs Polars: métodos básicos

Vamos a cargar los dos datasets en memoria para comparar la sintaxís e ir conociendo polars.

Aquí vamos a utilizar la `API de Eager mode de polars` y por este motivo vamos a utilizar el método de `read_parquet` de polars.

Más adelante veremos los `LazyFrames` de polars.

In [25]:
TRIP2401

'/Users/nicolaepopescul/code/polanyt/output/yellow_tripdata_2024-01.parquet'

In [27]:
# cargar el dataset
pddf = pd.read_parquet(TRIP2401)

In [29]:
# polars
pldf = pl.read_parquet(TRIP2401)

In [30]:
# hacer el shape del dataset
# pandas
pddf.shape

(2964624, 19)

In [31]:
# polars
pldf.shape

(2964624, 19)

In [41]:
# hacer el describe del dataset
# pandas
pddf.describe()

,VendorID,TpepPickupDatetime,TpepDropoffDatetime,PassengerCount,TripDistance,RatecodeID,PULocationID,DOLocationID,PaymentType,FareAmount,Extra,MtaTax,TipAmount,TollsAmount,ImprovementSurcharge,TotalAmount,CongestionSurcharge,AirportFee
count,2964624.000,2964624,2964624,2824462.000,2964624.000,2824462.000,2964624.000,2964624.000,2964624.000,2964624.000,2964624.000,2964624.000,2964624.000,2964624.000,2964624.000,2964624.000,2824462.000,2824462.000
mean,1.754,2024-01-17 00:46:36.431092,2024-01-17 01:02:13.208130,1.339,3.652,2.069,166.018,165.117,1.161,18.175,1.452,0.483,3.336,0.527,0.976,26.802,2.256,0.141
min,1.000,2002-12-31 22:59:39,2002-12-31 23:05:41,0.000,0.000,1.000,1.000,1.000,0.000,-899.000,-7.500,-0.500,-80.000,-80.000,-1.000,-900.000,-2.500,-1.750
25%,2.000,2024-01-09 15:59:19.750000,2024-01-09 16:16:23,1.000,1.000,1.000,132.000,114.000,1.000,8.600,0.000,0.500,1.000,0.000,1.000,15.380,2.500,0.000
50%,2.000,2024-01-17 10:45:37.500000,2024-01-17 11:03:51.500000,1.000,1.680,1.000,162.000,162.000,1.000,12.800,1.000,0.500,2.700,0.000,1.000,20.100,2.500,0.000
75%,2.000,2024-01-24 18:23:52.250000,2024-01-24 18:40:29,1.000,3.110,1.000,234.000,234.000,1.000,20.500,2.500,0.500,4.120,0.000,1.000,28.560,2.500,0.000
max,6.000,2024-02-01 00:01:15,2024-02-02 13:56:52,9.000,312722.300,99.000,265.000,265.000,4.000,5000.000,14.250,4.000,428.000,115.920,1.000,5000.000,2.500,1.750
std,0.433,NaN,NaN,0.850,225.463,9.823,63.624,69.315,0.581,18.950,1.804,0.118,3.897,2.128,0.218,23.386,0.823,0.488


In [42]:
# polars
pldf.describe()

statistic,VendorID,TpepPickupDatetime,TpepDropoffDatetime,PassengerCount,TripDistance,RatecodeID,StoreAndFwdFlag,PULocationID,DOLocationID,PaymentType,FareAmount,Extra,MtaTax,TipAmount,TollsAmount,ImprovementSurcharge,TotalAmount,CongestionSurcharge,AirportFee
str,f64,str,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",2964624.000,"""2964624""","""2964624""",2824462.000,2964624.000,2824462.000,"""2824462""",2964624.000,2964624.000,2964624.000,2964624.000,2964624.000,2964624.000,2964624.000,2964624.000,2964624.000,2964624.000,2824462.000,2824462.000
"""null_count""",0.000,"""0""","""0""",140162.000,0.000,140162.000,"""140162""",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,140162.000,140162.000
"""mean""",1.754,"""2024-01-17 00:46:36.431093""","""2024-01-17 01:02:13.208130""",1.339,3.652,2.069,null,166.018,165.117,1.161,18.175,1.452,0.483,3.336,0.527,0.976,26.802,2.256,0.141
"""std""",0.433,null,null,0.850,225.463,9.823,null,63.624,69.315,0.581,18.950,1.804,0.118,3.897,2.128,0.218,23.386,0.823,0.488
"""min""",1.000,"""2002-12-31 22:59:39""","""2002-12-31 23:05:41""",0.000,0.000,1.000,"""N""",1.000,1.000,0.000,-899.000,-7.500,-0.500,-80.000,-80.000,-1.000,-900.000,-2.500,-1.750
"""25%""",2.000,"""2024-01-09 15:59:20""","""2024-01-09 16:16:23""",1.000,1.000,1.000,null,132.000,114.000,1.000,8.600,0.000,0.500,1.000,0.000,1.000,15.380,2.500,0.000
"""50%""",2.000,"""2024-01-17 10:45:38""","""2024-01-17 11:03:52""",1.000,1.680,1.000,null,162.000,162.000,1.000,12.800,1.000,0.500,2.700,0.000,1.000,20.100,2.500,0.000
"""75%""",2.000,"""2024-01-24 18:23:52""","""2024-01-24 18:40:29""",1.000,3.110,1.000,null,234.000,234.000,1.000,20.500,2.500,0.500,4.120,0.000,1.000,28.560,2.500,0.000
"""max""",6.000,"""2024-02-01 00:01:15""","""2024-02-02 13:56:52""",9.000,312722.300,99.000,"""Y""",265.000,265.000,4.000,5000.000,14.250,4.000,428.000,115.920,1.000,5000.000,2.500,1.750


In [43]:
# hacer el info del dataset
# pandas
pddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 19 columns):
 #   Column                Dtype         
---  ------                -----         
 0   VendorID              int64         
 1   TpepPickupDatetime    datetime64[us]
 2   TpepDropoffDatetime   datetime64[us]
 3   PassengerCount        float64       
 4   TripDistance          float64       
 5   RatecodeID            float64       
 6   StoreAndFwdFlag       object        
 7   PULocationID          int64         
 8   DOLocationID          int64         
 9   PaymentType           int64         
 10  FareAmount            float64       
 11  Extra                 float64       
 12  MtaTax                float64       
 13  TipAmount             float64       
 14  TollsAmount           float64       
 15  ImprovementSurcharge  float64       
 16  TotalAmount           float64       
 17  CongestionSurcharge   float64       
 18  AirportFee            float64       
dtype

In [44]:
# polars
pldf.schema

OrderedDict([('VendorID', Int64),
             ('TpepPickupDatetime', Datetime(time_unit='us', time_zone=None)),
             ('TpepDropoffDatetime', Datetime(time_unit='us', time_zone=None)),
             ('PassengerCount', Float64),
             ('TripDistance', Float64),
             ('RatecodeID', Float64),
             ('StoreAndFwdFlag', String),
             ('PULocationID', Int64),
             ('DOLocationID', Int64),
             ('PaymentType', Int64),
             ('FareAmount', Float64),
             ('Extra', Float64),
             ('MtaTax', Float64),
             ('TipAmount', Float64),
             ('TollsAmount', Float64),
             ('ImprovementSurcharge', Float64),
             ('TotalAmount', Float64),
             ('CongestionSurcharge', Float64),
             ('AirportFee', Float64)])

In [45]:
# sacar las columnas del dataset
# pandas
pddf.columns

Index(['VendorID', 'TpepPickupDatetime', 'TpepDropoffDatetime',
       'PassengerCount', 'TripDistance', 'RatecodeID', 'StoreAndFwdFlag',
       'PULocationID', 'DOLocationID', 'PaymentType', 'FareAmount', 'Extra',
       'MtaTax', 'TipAmount', 'TollsAmount', 'ImprovementSurcharge',
       'TotalAmount', 'CongestionSurcharge', 'AirportFee'],
      dtype='object')

In [46]:
# polars
pldf.columns

['VendorID',
 'TpepPickupDatetime',
 'TpepDropoffDatetime',
 'PassengerCount',
 'TripDistance',
 'RatecodeID',
 'StoreAndFwdFlag',
 'PULocationID',
 'DOLocationID',
 'PaymentType',
 'FareAmount',
 'Extra',
 'MtaTax',
 'TipAmount',
 'TollsAmount',
 'ImprovementSurcharge',
 'TotalAmount',
 'CongestionSurcharge',
 'AirportFee']

In [47]:
# ver las primeras 5 filas (head)
# pandas
pddf.head()

,VendorID,TpepPickupDatetime,TpepDropoffDatetime,PassengerCount,TripDistance,RatecodeID,StoreAndFwdFlag,PULocationID,DOLocationID,PaymentType,FareAmount,Extra,MtaTax,TipAmount,TollsAmount,ImprovementSurcharge,TotalAmount,CongestionSurcharge,AirportFee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.000,1.720,1.000,N,186,79,2,17.700,1.000,0.500,0.000,0.000,1.000,22.700,2.500,0.000
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.000,1.800,1.000,N,140,236,1,10.000,3.500,0.500,3.750,0.000,1.000,18.750,2.500,0.000
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.000,4.700,1.000,N,236,79,1,23.300,3.500,0.500,3.000,0.000,1.000,31.300,2.500,0.000
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.000,1.400,1.000,N,79,211,1,10.000,3.500,0.500,2.000,0.000,1.000,17.000,2.500,0.000
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.000,0.800,1.000,N,211,148,1,7.900,3.500,0.500,3.200,0.000,1.000,16.100,2.500,0.000


In [48]:
# polars
pldf.head()

VendorID,TpepPickupDatetime,TpepDropoffDatetime,PassengerCount,TripDistance,RatecodeID,StoreAndFwdFlag,PULocationID,DOLocationID,PaymentType,FareAmount,Extra,MtaTax,TipAmount,TollsAmount,ImprovementSurcharge,TotalAmount,CongestionSurcharge,AirportFee
i64,datetime[μs],datetime[μs],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.000,1.720,1.000,"""N""",186,79,2,17.700,1.000,0.500,0.000,0.000,1.000,22.700,2.500,0.000
1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.000,1.800,1.000,"""N""",140,236,1,10.000,3.500,0.500,3.750,0.000,1.000,18.750,2.500,0.000
1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.000,4.700,1.000,"""N""",236,79,1,23.300,3.500,0.500,3.000,0.000,1.000,31.300,2.500,0.000
1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.000,1.400,1.000,"""N""",79,211,1,10.000,3.500,0.500,2.000,0.000,1.000,17.000,2.500,0.000
1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.000,0.800,1.000,"""N""",211,148,1,7.900,3.500,0.500,3.200,0.000,1.000,16.100,2.500,0.000


In [50]:
# sacar una muestra del dataset
# pandas
pddf.sample(10)

,VendorID,TpepPickupDatetime,TpepDropoffDatetime,PassengerCount,TripDistance,RatecodeID,StoreAndFwdFlag,PULocationID,DOLocationID,PaymentType,FareAmount,Extra,MtaTax,TipAmount,TollsAmount,ImprovementSurcharge,TotalAmount,CongestionSurcharge,AirportFee
2328198,1,2024-01-26 19:33:01,2024-01-26 19:44:23,1.000,1.800,1.000,N,237,238,1,11.400,5.000,0.500,3.600,0.000,1.000,21.500,2.500,0.000
1434796,2,2024-01-17 14:54:09,2024-01-17 14:59:12,1.000,0.460,1.000,N,163,163,2,6.500,0.000,0.500,0.000,0.000,1.000,10.500,2.500,0.000
467816,2,2024-01-06 15:20:32,2024-01-06 15:27:22,1.000,0.620,1.000,N,162,141,2,7.900,0.000,0.500,0.000,0.000,1.000,11.900,2.500,0.000
2778461,2,2024-01-31 16:20:38,2024-01-31 16:29:45,1.000,1.480,1.000,N,236,262,1,10.700,2.500,0.500,3.440,0.000,1.000,20.640,2.500,0.000
1395,2,2024-01-01 00:56:35,2024-01-01 01:04:35,1.000,1.760,1.000,N,43,24,1,10.700,1.000,0.500,3.920,0.000,1.000,19.620,2.500,0.000
1833883,2,2024-01-21 15:42:58,2024-01-21 15:50:42,2.000,1.340,1.000,N,140,229,1,10.000,0.000,0.500,1.400,0.000,1.000,15.400,2.500,0.000
1562419,2,2024-01-18 18:19:32,2024-01-18 18:26:09,1.000,0.910,1.000,N,234,114,1,7.900,2.500,0.500,2.000,0.000,1.000,16.400,2.500,0.000
2449070,2,2024-01-27 22:41:18,2024-01-27 22:51:10,3.000,1.440,1.000,N,163,170,2,10.700,1.000,0.500,0.000,0.000,1.000,15.700,2.500,0.000
354329,2,2024-01-05 12:37:25,2024-01-05 12:47:53,1.000,1.180,1.000,N,137,164,1,10.700,0.000,0.500,5.000,0.000,1.000,19.700,2.500,0.000
1060978,2,2024-01-13 11:44:34,2024-01-13 11:47:35,1.000,0.780,1.000,N,239,238,1,5.800,0.000,0.500,1.960,0.000,1.000,11.760,2.500,0.000


In [51]:
# polars
pldf.sample(10)

VendorID,TpepPickupDatetime,TpepDropoffDatetime,PassengerCount,TripDistance,RatecodeID,StoreAndFwdFlag,PULocationID,DOLocationID,PaymentType,FareAmount,Extra,MtaTax,TipAmount,TollsAmount,ImprovementSurcharge,TotalAmount,CongestionSurcharge,AirportFee
i64,datetime[μs],datetime[μs],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2024-01-16 12:26:06,2024-01-16 12:46:48,1.000,1.700,1.000,"""N""",163,140,1,17.000,2.500,0.500,5.250,0.000,1.000,26.250,2.500,0.000
2,2024-01-12 15:18:47,2024-01-12 15:28:46,1.000,1.280,1.000,"""N""",68,48,1,10.700,0.000,0.500,2.940,0.000,1.000,17.640,2.500,0.000
2,2024-01-27 00:01:55,2024-01-27 00:07:31,1.000,0.020,1.000,"""N""",48,48,4,-6.500,-1.000,-0.500,0.000,0.000,-1.000,-11.500,-2.500,0.000
2,2024-01-18 17:17:04,2024-01-18 17:31:01,1.000,1.470,1.000,"""N""",237,161,1,13.500,2.500,0.500,5.000,0.000,1.000,25.000,2.500,0.000
2,2024-01-14 08:31:50,2024-01-14 08:38:47,1.000,0.610,1.000,"""N""",68,246,1,7.900,0.000,0.500,2.380,0.000,1.000,14.280,2.500,0.000
2,2024-01-28 14:03:07,2024-01-28 14:09:51,1.000,0.750,1.000,"""N""",234,107,1,7.900,0.000,0.500,2.380,0.000,1.000,14.280,2.500,0.000
2,2024-01-17 18:21:59,2024-01-17 18:31:14,1.000,0.920,1.000,"""N""",236,237,2,10.000,2.500,0.500,0.000,0.000,1.000,16.500,2.500,0.000
2,2024-01-29 20:13:02,2024-01-29 20:17:02,1.000,0.720,1.000,"""N""",141,237,2,5.800,1.000,0.500,0.000,0.000,1.000,10.800,2.500,0.000
2,2024-01-16 22:51:49,2024-01-16 22:59:12,1.000,0.930,1.000,"""N""",163,233,1,8.600,1.000,0.500,1.000,0.000,1.000,14.600,2.500,0.000


In [54]:
no_nulls = pddf.dropna()

In [58]:
no_nulls.head()

,VendorID,TpepPickupDatetime,TpepDropoffDatetime,PassengerCount,TripDistance,RatecodeID,StoreAndFwdFlag,PULocationID,DOLocationID,PaymentType,FareAmount,Extra,MtaTax,TipAmount,TollsAmount,ImprovementSurcharge,TotalAmount,CongestionSurcharge,AirportFee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.000,1.720,1.000,N,186,79,2,17.700,1.000,0.500,0.000,0.000,1.000,22.700,2.500,0.000
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.000,1.800,1.000,N,140,236,1,10.000,3.500,0.500,3.750,0.000,1.000,18.750,2.500,0.000
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.000,4.700,1.000,N,236,79,1,23.300,3.500,0.500,3.000,0.000,1.000,31.300,2.500,0.000
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.000,1.400,1.000,N,79,211,1,10.000,3.500,0.500,2.000,0.000,1.000,17.000,2.500,0.000
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.000,0.800,1.000,N,211,148,1,7.900,3.500,0.500,3.200,0.000,1.000,16.100,2.500,0.000


In [63]:
no_nulls["FareAmountx2"] = no_nulls["FareAmount"] * 2

In [60]:
import numpy as np

In [61]:
no_nulls.groupby(["StoreAndFwdFlag"])[["PassengerCount", "TripDistance"]].agg(
    [np.mean, np.median, len]
)

PassengerCount                 TripDistance                
                          mean median      len         mean median      len
StoreAndFwdFlag                                                            
N                        1.340  1.000  2813126        3.255  1.670  2813126
Y                        1.214  1.000    11336        2.935  1.600    11336

In [71]:
(
    no_nulls.groupby(["StoreAndFwdFlag"])[["PassengerCount", "TripDistance"]].agg(
        [np.mean, np.median, len]
    )
)

PassengerCount                 TripDistance                
                          mean median      len         mean median      len
StoreAndFwdFlag                                                            
N                        1.340  1.000  2813126        3.255  1.670  2813126
Y                        1.214  1.000    11336        2.935  1.600    11336

In [70]:
(
    pddf.dropna()
    .assign(FareAmount2=lambda df: df["FareAmount"] * 2)
    .groupby(["StoreAndFwdFlag"])
    .agg(
        Count=("PassengerCount", sum),
        Mean=("PassengerCount", np.mean),
        Median=("TripDistance", np.median),
    )
)

,Count,Mean,Median
StoreAndFwdFlag,,,
N,3768991.000,1.340,1.670
Y,13757.000,1.214,1.600


# .with_columns, .select, .filter ...

In [74]:
# sacar la media de la column PassengerCount
# pandas
(pddf["PassengerCount"].mean())

1.3392808966805005

In [78]:
# polars
(
    pldf.select(pl.col("PassengerCount")).mean()  # contexto  # expresión
)

PassengerCount
f64
1.339


### `Came for speed, stayed for the syntax.`

Dentro de polars hay una sintaxis específica que se recomienda seguir.

Vamos a ver algunas "peculiaridades de la sintaxis de polars".

* Para seleccionar una columna debemos usar `.select`

En polars al `.select` se le conoce como `contexto` y es el **"entorno"** donde se van a evaluar `expresiones de polars`.

Existen otros contextos dentro de polars. Por ejemplo: `group_by`, `select`, `filter` son todo contextos de polars.

Por el otro lado, podemos pensar en `expresiones` como operaciones que queremos que se evaluen y ejecuten dentro de un contexto. Por ejemplo: `sum`, `min`, `max` etc. son todo expresiones que se pueden evaluar dentro de determinados contextos.

In [79]:
# sacar la media de la column PassengerCount

# polars
(
    pldf.select(pl.col("PassengerCount")).mean()  # contexto  # expresión
)

PassengerCount
f64
1.339


* Si queremos crear una nueva columna debemos usar `.with_columns`

In [80]:
pddf.head()

,VendorID,TpepPickupDatetime,TpepDropoffDatetime,PassengerCount,TripDistance,RatecodeID,StoreAndFwdFlag,PULocationID,DOLocationID,PaymentType,FareAmount,Extra,MtaTax,TipAmount,TollsAmount,ImprovementSurcharge,TotalAmount,CongestionSurcharge,AirportFee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.000,1.720,1.000,N,186,79,2,17.700,1.000,0.500,0.000,0.000,1.000,22.700,2.500,0.000
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.000,1.800,1.000,N,140,236,1,10.000,3.500,0.500,3.750,0.000,1.000,18.750,2.500,0.000
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.000,4.700,1.000,N,236,79,1,23.300,3.500,0.500,3.000,0.000,1.000,31.300,2.500,0.000
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.000,1.400,1.000,N,79,211,1,10.000,3.500,0.500,2.000,0.000,1.000,17.000,2.500,0.000
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.000,0.800,1.000,N,211,148,1,7.900,3.500,0.500,3.200,0.000,1.000,16.100,2.500,0.000


In [100]:
# calcular el número total de nulos que hay en la columna de PassengerCount

# usar el assign de pandas
# pandas

(
    pddf.assign(
        NullValues=lambda df: df["PassengerCount"].isnull(),
        NanValues=lambda df: df["PassengerCount"].isna(),
    )[["NullValues", "NanValues"]].sum()
)

NullValues    140162
NanValues     140162
dtype: int64

Polars distingue entre `nulls` y `nans`.

Los `nulls` son valores `missing` dentro de nuestro dataset.

Los `nans` por el otro lado son `not a number`. Por ejemplo: `infinito`.

In [99]:
# calcular el número total de nulos y nans que hay en la columna de PassengerCount

(
    pldf.select(pl.col("PassengerCount"))
    .with_columns(
        NullValues=pl.col("PassengerCount").is_null(),
        NanValues=pl.col("PassengerCount").is_nan(),
        # pl.col("PassengerCount").is_null().alias("NanValues"),
    )
    .select(pl.col(["NullValues", "NanValues"]))
    .sum()
)

# filtrar PassengerCount
# usar el assign
# drop PassengerCount
# pandas

NullValues,NanValues
u32,u32
140162,0


In [ ]:
# usar with_columns
# select y sum
# polars

In [ ]:
# calcular el número total de nulos que hay por columna

# usar with_columns con pl.all()
# melt
# rename
# filter
# polars

* Cuando queremos filtrar un dataframe podemos utilizar el método de `.filter` de polars.

In [ ]:
# calcular el min, mean y max por StoreAndFwdFlag cuando VendorId == 1 y no hay nulos en PassengerCount
# filtrar primero
# groupby
# agg
# sort
# reset
# pandas

In [ ]:
# filter
# group_by
# agg
# sort
# polars

# .group_by 

In [ ]:
# calcular el TotalTripDistance por día de la semana entre 2024-1-1 y 2024-2-1 (inclusive) y cuando
# no hay nulos en PassengerCount

# filter
# assing DayOfWeek
# groupby
# agg
# sort_values
# pandas

In [ ]:
# filter
# with_columns pl.col is_between
# with_columns DayOfWeek
# group_by
# agg
# sort
# polars

# Plots

Tanto pandas como polars tienen capacidad para realizar gráficos desde la propia librería.

Pandas utiliza [Matplotlib](https://matplotlib.org/) mientras que polars utiliza [hvPlot](https://hvplot.holoviz.org/).

In [ ]:
# hacer un bar plot de la distancia media por VendorId

# groupby
# agg
# reset y sort
# Vendor, MeanTripDistance
# plot
# pandas

In [ ]:
# group_by
# agg
# sort
# select pl.col().cast(str)
# hvplot
# polars

# Lazy Mode

Cuando utilizamos cualquier de los métodos de polars que empiezan por `scan_` estamos usando el `Lazy Mode` de polars.

Esto implica que: 
1. No estamos cargando el dataset en memoria.
2. El query planner puede hacer algunas optimizaciones de nuestro código.

In [ ]:
# mostrar el pl.scan

In [ ]:
# ver el naive query plan

In [ ]:
# ver el naive query plan de la siguiente operación

# crear un df con 3 columnas
# TpepPickupDatetime, DayOfWeek, FlagPassengerCount
# cuando TpepPickupDatetime está entre 2024-1-1 y 2024-2-1
# no hay nulos en PassengerCount
# y cuando el PassengerCount es superior a >= 3

# filter
# with DayOfWeek y FlagPassengerCount >= 3
# select

In [ ]:
# mostrar el query plan optimizado
# show_graph

In [ ]:
# mostrar el método de collect para ejecutar y "recolectar" los resultados
# sacar el total de nulos que hay entre todas las columnas

# select pl.all().is_null()
# sum
# collect

In [ ]:
COLS = [
    "VendorID",
    "TpepPickupDatetime",
    "TpepDropoffDatetime",
    "PassengerCount",
    "TripDistance",
    "RatecodeID",
    "StoreAndFwdFlag",
    "PULocationID",
    "DOLocationID",
    "PaymentType",
    "FareAmount",
    "Extra",
    "MtaTax",
    "TipAmount",
    "TollsAmount",
    "ImprovementSurcharge",
    "TotalAmount",
    "CongestionSurcharge",
    "AirportFee",
]

Con LazyFrames de `polars` podemos procesar **194.457.948 de registros.**

In [ ]:
# mostrar como podemos procesar nuestro dataset de 194.457.948 de registros

# select pl.col(COLS)
# collect
# shape

El tamaño total de nuestro dataset en memoria es de **26GB**.

In [ ]:
# calcular el tamaño de nuestro dataset en memoria

# select pl.col(COLS)
# collect
# estimated_size()

In [ ]:
# hacer el head del dataset

In [ ]:
# select TpepPickupDatetime y PassengerCount

# nos quedamos sólo con aquellas filas que no tienen ningún nulo a nivel de fila
# ~pl.all_horizontal(pl.all().is_null())

# filter TpepPickupDatetime entre 2020-12-31 y 2024-3-1
# sort TpepPickupDatetime
# group_by_dynamic(index="TpepPickupDatetime", every)

# agg (TotalPassengers)

# collect
# sort
# hvplot

In [ ]:
# 2.49 s ± 190 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
# %%timeit

# Conclusión

Tras este breve workshop, podemos extrar algunas conclusiones:
1. Tanto pandas como polars son excelentes librerás para hacer análisis de datos.
2. Pandas es una excelente opción siempre y cuando tenemos datasets pequeños y que caben en memoria.
3. Pandas también es una librería mucho más madura que polars y que tiene mucha documentación y ejemplos.
4. Polars es una de las mejores opciones cuando tenemos que trabajar con dataset grandes que no caben en memoria y no podemos utilizar tecnologías Big Data.
5. Polars está en fase activa de desarrollo.

Recomendación personal: si vas a empezar un nuevo proyecto utiliza polars (aprender la librería etc).

No hagan "grandes migraciones de código" porque es muy costoso a nivel de tiempo.

Si caso de echar en falta algo de pandas, siempre pueden utilizar `.to_pandas()`

In [ ]:
# polars to pandas